In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression , RidgeClassifier, Huber
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier , ExtraTreesClassifier , AdaBoostClassifier
from sklearn.model_selection import KFold ,StratifiedKFold
from sklearn.metrics import log_loss
import warnings 
warnings.filterwarnings('ignore')

In [4]:
train = train[train['time']<2*1e6]
train = train[train['time']!=1804691]
train = train[train['Store_Ratio']>0]
# train.shape
train.head()

,Store_Ratio,Basket_Ratio,Category_1,Store_Score,Category_2,Store_Presence,Score_1,Score_2,Score_3,Score_4,time,popularity
0,0.407,0.00380,2,-35.865,1,0.9920,0.944,0.0988,0.1100,113.911,189125,4
1,0.234,0.10500,0,-19.884,1,0.9440,0.900,0.1290,0.0382,76.332,186513,4
2,0.668,0.72600,9,-7.512,1,0.0387,0.000,0.1340,0.4530,124.075,172143,4
3,0.184,0.00561,4,-34.357,1,0.8490,0.931,0.1110,0.0641,79.037,184000,5
4,0.231,0.13100,6,-22.842,1,0.9370,0.000,0.1090,0.0677,109.560,186507,3


In [5]:
train.shape

(18205, 12)

(18189, 12)

In [7]:
train.popularity.value_counts()

4    15229
3     2165
5      472
1      323
Name: popularity, dtype: int64

In [8]:
X = train.drop('popularity',axis=1)
Y = train['popularity']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_ = scaler.fit_transform(X)
X = pd.DataFrame(X_,columns=X.columns)

folds = StratifiedKFold(n_splits=10,shuffle=True ,random_state=42)

oof_preds = []
oof_scores = []

for i , (train_idx,test_idx) in enumerate(folds.split(X,Y)):
  train_set = (X.iloc[train_idx],Y.iloc[train_idx])
  test_set = (X.iloc[test_idx],Y.iloc[test_idx])

  model = CatBoostClassifier(loss_function='MultiClass', 
                         eval_metric='TotalF1', 
                         depth=7,
                         verbose=0
)
  model.fit(*train_set)
  preds = model.predict_proba(test_set[0])
  score = log_loss(test_set[1],preds)
  
  oof_pred_fold = model.predict_proba(scaler.transform(test))
  oof_preds.append(oof_pred_fold)

  print(score)
  oof_scores.append(score)

print("Mean OOF Score :",np.mean(oof_scores))
final_preds = np.mean(np.array(oof_preds),axis=0)

0.40622963042251287
0.4147608064892967
0.4083242484803806
0.41242385360564
0.3933343406766553
0.3833319521429497
0.3776756966009344
0.40536682578445876
0.38166896790431204
0.3952124277656655
Mean OOF Score : 0.3978328749872806


In [9]:
temp = pd.DataFrame(final_preds)
temp.head()


,0,1,2,3
0,0.018397,0.714715,0.206470,0.060419
1,0.003133,0.017828,0.976881,0.002158
2,0.000508,0.034272,0.957876,0.007344
3,0.002176,0.024373,0.965493,0.007958
4,0.000220,0.003145,0.994387,0.002248


In [14]:
temp.columns

RangeIndex(start=0, stop=4, step=1)

In [16]:
sub = pd.DataFrame()
sub['0'] = 0
sub['1'] = temp[0]
sub['2'] = temp[1]
sub['3'] = temp[2]
sub['4'] = temp[3]
sub = sub.fillna(0)

In [19]:
sub.head()

,0,1,2,3,4
0,0.0,0.018397,0.714715,0.206470,0.060419
1,0.0,0.003133,0.017828,0.976881,0.002158
2,0.0,0.000508,0.034272,0.957876,0.007344
3,0.0,0.002176,0.024373,0.965493,0.007958
4,0.0,0.000220,0.003145,0.994387,0.002248


In [20]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''
for i in test.columns:
    temp1=temp1+i+train[i].astype(str)
train['ALL']=temp1
temp2=''
for i in test.columns:
    temp2=temp2+i+test[i].astype(str)
test['ALL']=temp2
encoding=train.groupby('ALL')['popularity'].median()
y_from_train=test['ALL'].map(encoding)
temp = pd.DataFrame()
temp['pop'] = y_from_train
temp['pop'] = temp['pop'].fillna(-999)
x = temp[temp['pop']!=-999]
for i in x.index.values:
#     print(x.iloc[i].values[0])
    try:
        if x.loc[i].values[0] ==0:
            sub.loc[i] =[1.0,0.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==1:
            sub.loc[i] =[0.0,1.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==3:
            sub.loc[i] =[0.0,0.0,1.0,0.0,0.0]
        elif x.loc[i].values[0] ==4:
            sub.loc[i] =[0.0,0.0,0.0,1.0,0.0]
        elif x.loc[i].values[0] ==5:
            sub.loc[i] =[0.0,0.0,0.0,0.0,1.0]
        else:
            print('.',end='')
    except:
        print(i)
        print(x.loc[i].values[0])

In [23]:
sub.head()

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.003133,0.017828,0.976881,0.002158
2,0.0,0.000508,0.034272,0.957876,0.007344
3,0.0,0.002176,0.024373,0.965493,0.007958
4,0.0,0.000000,0.000000,1.000000,0.000000


In [26]:
dummy = test[test['Store_Ratio']==0]
dummy.index

Int64Index([358, 2873, 5982, 5984, 6306, 7565, 7761, 8615, 8829, 11152, 12010], dtype='int64')

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.003133,0.017828,0.976881,0.002158
2,0.0,0.000508,0.034272,0.957876,0.007344
3,0.0,0.002176,0.024373,0.965493,0.007958
4,0.0,0.000000,0.000000,1.000000,0.000000


In [30]:
sub.to_csv('submission_2501_v1.csv',index=False)

In [33]:
from sklearn.model_selection import train_test_split

In [36]:
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, Y, stratify = train.popularity,test_size=0.1)

In [39]:
# !pip install optuna
import optuna.integration.lightgbm as lgb
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_test, label=y_test)

params = {
        "metric": "auc",
        "verbosity": -1,
        "boosting_type": "gbdt",
}

model = lgb.train(
        params, dtrain, valid_sets=[dtrain, dval], verbose_eval=100, early_stopping_rounds=100
    )

prediction = np.rint(model.predict(X_test, num_iteration=model.best_iteration))
accuracy = accuracy_score(y_test, prediction)

best_params = model.params
print("Best params:", best_params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in best_params.items():
  print("    {}: {}".format(key, value))


[I 2021-01-25 14:21:47,020] A new study created in memory with name: no-name-a2b30359-37f3-4a43-a9a0-fe78bc2262a9
feature_fraction, val_score: -inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


feature_fraction, val_score: 1.000000:  14%|#4        | 1/7 [00:00<00:02,  2.07it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


feature_fraction, val_score: 1.000000:  29%|##8       | 2/7 [00:00<00:02,  2.44it/s]

...

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


min_data_in_leaf, val_score: 1.000000:  80%|########  | 4/5 [00:01<00:00,  2.21it/s]

Training until validation scores don't improve for 100 rounds
[100]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[1]	training's auc: 1	valid_1's auc: 1


min_data_in_leaf, val_score: 1.000000: 100%|##########| 5/5 [00:02<00:00,  1.81it/s]


Best params: {'metric': 'auc', 'verbosity': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'num_leaves': 31, 'feature_fraction': 0.8999999999999999, 'bagging_fraction': 1.0, 'bagging_freq': 0, 'min_child_samples': 20}
  Accuracy = 0.8372732270478285
  Params: 
    metric: auc
    verbosity: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 0.0
    lambda_l2: 0.0
    num_leaves: 31
    feature_fraction: 0.8999999999999999
    bagging_fraction: 1.0
    bagging_freq: 0
    min_child_samples: 20


In [54]:
params = {
    'metric': 'multi_logloss',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'feature_pre_filter': False,
    'lambda_l1': 0.0,
    'lambda_l2': 0.0,
    'num_leaves': 31,
    'feature_fraction': 0.8999999999999999,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 20}

In [55]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(**params)

In [56]:
model.fit(X_train,y_train)

LGBMClassifier(bagging_fraction=1.0, bagging_freq=0,
               feature_fraction=0.8999999999999999, feature_pre_filter=False,
               lambda_l1=0.0, lambda_l2=0.0, metric='multi_logloss',
               verbosity=-1)

In [57]:
preds_val = model.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,preds_val))

0.8565145684442002


In [58]:
final_preds = model.predict_proba(test.drop(columns=['ALL'],axis=1))
final_preds

array([[0.00443583, 0.13178849, 0.85692523, 0.00685045],
       [0.00485916, 0.19611271, 0.78708085, 0.01194728],
       [0.01910739, 0.17706252, 0.78872045, 0.01510963],
       ...,
       [0.006784  , 0.18438983, 0.79955049, 0.00927568],
       [0.01462067, 0.07277659, 0.89405947, 0.01854327],
       [0.00802544, 0.15986065, 0.8055979 , 0.02651601]])

In [59]:
temp = pd.DataFrame(final_preds)
sub = pd.DataFrame()
sub['0'] = 0
sub['1'] = temp[0]
sub['2'] = temp[1]
sub['3'] = temp[2]
sub['4'] = temp[3]


In [61]:
sub = sub.fillna(0)

In [62]:
#there are duplicates between test and train, for those rows we can directly use train target values
#storing the target values of common rows in test and train
temp1=''
for i in test.columns:
    temp1=temp1+i+train[i].astype(str)
train['ALL']=temp1
temp2=''
for i in test.columns:
    temp2=temp2+i+test[i].astype(str)
test['ALL']=temp2
encoding=train.groupby('ALL')['popularity'].median()
y_from_train=test['ALL'].map(encoding)
temp = pd.DataFrame()
temp['pop'] = y_from_train
temp['pop'] = temp['pop'].fillna(-999)
x = temp[temp['pop']!=-999]
for i in x.index.values:
#     print(x.iloc[i].values[0])
    try:
        if x.loc[i].values[0] ==0:
            sub.loc[i] =[1.0,0.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==1:
            sub.loc[i] =[0.0,1.0,0.0,0.0,0.0]
        elif x.loc[i].values[0] ==3:
            sub.loc[i] =[0.0,0.0,1.0,0.0,0.0]
        elif x.loc[i].values[0] ==4:
            sub.loc[i] =[0.0,0.0,0.0,1.0,0.0]
        elif x.loc[i].values[0] ==5:
            sub.loc[i] =[0.0,0.0,0.0,0.0,1.0]
        else:
            print('.',end='')
    except:
        print(i)
        print(x.loc[i].values[0])

In [63]:
dummy = test[test['Store_Ratio']==0]
dummy.index
for i in dummy.index.values:
    sub.iloc[i] =[1.0,0.0,0.0,0.0,0.0]
sub.head()

,0,1,2,3,4
0,0.0,0.000000,1.000000,0.000000,0.000000
1,0.0,0.004859,0.196113,0.787081,0.011947
2,0.0,0.019107,0.177063,0.788720,0.015110
3,0.0,0.007202,0.188808,0.794248,0.009742
4,0.0,0.000000,0.000000,1.000000,0.000000


In [64]:
np.sum(sub['0'])

11.0

In [65]:
sub.to_csv('sub_2501_v2_lgb.csv',index=False)